<a href="https://colab.research.google.com/github/kingao12/-_AI-Data-_-1/blob/main/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [120]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [121]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 정규식을 사용하는 라이브러리
import re

In [122]:
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/naver_news_content_20240720.xlsx')
df

,뉴스제목,본문,날짜
0,"""BTS 전역만 기다렸는데""…'설상가상' 하이브 개미들 '비명'..",'어도어 사태'·표절 의혹에 하이브 투자심리 위축\n2분기 실적도 부진할 것으로 예...,2024.07.20. 오후 3:48
1,"""트럼프 당선 시 비트코인 7만달러 갈 것""",암호화폐 A to Z\n미국 공화당 대선 후보로 공식 지명된 도널드 트럼프 전 대통...,2024.07.20. 오후 8:02
2,"""트럼프가 한국 증시 색깔 바꿨다""…외국인 팔아치운 종목은","""트럼프 트레이드 명분으로 AI 쏠림 완화""\n美대선주자 '자국 우선주의' 경쟁에…...",2024.07.20. 오후 2:39
3,내주부터 국내 증시서 시행된다는 ‘이것’…‘주식 먹튀’ 사라질까,‘블록딜 사전 공시 의무화’ 시행\n시행 전 대거 블록딜 매물 유의\n\n\n[사진...,2024.07.20. 오후 7:08
4,"""15년만의 투자 황금기""…놓치면 '후회막심' [투자의 재발견]",[투자의 재발견]\n\n'알채권' 직접 담을까? 채권 ETF 살까?\n\n상반기 개...,2024.07.20. 오전 6:00
...,...,...,...
95,마곡 CP4 '원그로브' 준공 눈앞…공실 채우는 데 2년 걸릴 ..,"주변 대형 오피스 연이어 준공…'공급 폭탄'\n마곡 CP2블록, 이달 말 준공…CP...",2024.07.20. 오전 6:25
96,"트래블로그 VS 쏠트래블…해외특화카드 '양대산맥', 일본서 직접..","[우리만 아는 금융꿀팁]상품·서비스\n[편집자주] 금융, 이것만 읽으면 쉽습니다. ...",2024.07.20. 오전 7:51
97,버튼만 누르면 고통 없이 죽는 안락사 캡슐…이달 첫 사용 예고,【 앵커멘트 】\n인간의 죽음을 돕는 '안락사 캡슐'이 조만간 스위스에서 처음 사용...,2024.07.19. 오후 8:00
98,"""돈만 있으면 더 사는데"" 개미들 한숨…회장님 매수에 '들썩' ..","한미반도체, 주가 급락 후 반등\n경쟁사 등장했지만 독주 체제 이어질 것이란 기대감...",2024.06.09. 오전 11:01


In [123]:
!pip install kss

In [124]:
import kss

In [125]:
# soynlp: 한국어 텍스트 데이터를 분석하고 처리하는 라이브러리
!pip install soynlp

In [126]:
from soynlp.normalizer import *

In [127]:
!pip install konlpy

In [128]:
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.


In [129]:
from konlpy.tag import Mecab

In [130]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [131]:
tokenizer = Tokenizer()

In [132]:
# 불용어 사전 정의
stopwords = ['이하', '바로', '＠','👆', '↑', '※']

# 불용어 제거 함수
def delete_stopwords(context):
  preprocessed_text = []
  for text in context:
    text = [w for w in text.split(' ') if w not in stopwords] # text를 띄어쓰기로 구분하고 불용어사전에 없는 데이터만 리스트에 저장
    preprocessed_text.append(' '.join(text)) # 띄어쓰기를 적용하여 리스트에 추가
  return preprocessed_text

In [133]:
# HTML 태그를 제거하는 함수
# re.sub(r'정규식', '대체될 텍스트', '문자열'): 문자열에서 패턴을 검색하고, 해당 패턴을 다른 문자열로 대체하는 역할
def delete_html_tag(context):
    processed_context = []
    for text in context:
        text = re.sub(r'<[%>]+Ws+(?=<)|<[^>]+>', '', text).strip()  # text에서 정규식에 해당하는 조건이 True가 되면 ''(제거)
        if text:
          processed_context.append(text)

    return processed_context

In [134]:
# 문장을 분리하기 위한 함수
def sentence_separator(processed_context):
  splited_context = []

  for text in processed_context:
    text = text.strip()  # 양쪽 공백 제거
    if text:
      splited_text = kss.split_sentences(text)   # 문장 단위로 분리
      splited_context.extend(splited_text)
  return splited_context

In [135]:
# 정규식을 통해 이메일 제거하는 함수
def delete_email(context):
    preprocessed_text = []
    for text in context:
        text = re.sub(r'[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+', '', text).strip()
        if text:
            preprocessed_text.append(text)
    return preprocessed_text

In [136]:
# 정규식을 통한 해시태그 제거
def delete_hashtag(context):
  preprocessed_text = []
  for text in context:
    text = re.sub(r'#\S+', '', text).strip()
    if text:
      preprocessed_text.append(text)
  return preprocessed_text


In [137]:
# 정규식을 통한 링크 제거
def delete_copyright(context):
    re_patterns = [
        r'\<저작권자(\(c\)|©|(C)|(\(C\))).+?\>',
        r'(Copyrights)|(\(c\))|(\(C\))|©|(C)|'
    ]
    preprocessed_text = []
    for text in context:
        for re_pattern in re_patterns:  # 위 패턴들만큼 반복
            text = re.sub(re_pattern, "", text).strip()
        if text:
            preprocessed_text.append(text)
    return preprocessed_text

In [138]:
def delete_special_characters(context):
    preprocessed_text = []
    for text in context:
        text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text).strip()
        if text:
            preprocessed_text.append(text)
    return preprocessed_text

In [139]:
!pip install soynlp
from soynlp.normalizer import *

def normalize_spacing(context):
    normalized_text = []
    for text in context:
        text = normalize(text)
        normalized_text.append(text)
    return normalized_text

In [140]:
from konlpy.tag import Mecab

mecab = Mecab()

def extract_stems(context):
    stemmed_text = []
    for text in context:
        stemmed_sentence = []
        for word in mecab.morphs(text):
            stemmed_sentence.append(word[:-1])  # 어간 추출
        stemmed_text.append(' '.join(stemmed_sentence))
    return stemmed_text

In [141]:
# OrderedDict: 순서가 있는 딕셔너리(Dictionary) 타입
# 기존의 딕셔너리와는 달리 아이템이 추가된 순서를 기억
from collections import OrderedDict

# 중복된 문장을 제거하는 함수
def duplicated_sentence_normalizer(context):
  context = list(OrderedDict.fromkeys(context))
  return context

In [142]:
# 객체 생성
mecab = Mecab()
morphs = mecab.pos('아버지가방에들어가신다', join=False)

# 형태소 분석 필터링
# 명사(NN), 동사(V), 형용사(J)의 포함 여부에 따라 문장 필터링
# 주어진 텍스트에서 명사, 동사, 형용사가 모두 포함된 텍스트들만을 추출하여 반환
def morph_filter(context):
  NN_TAGS = ['NNG', 'NNP', 'NNB', 'NP']
  V_TAGS = ['VV', 'VA', 'VX', 'VCP', 'VCN', 'XSN', 'XSA', 'XSV']
  J_TAGS = ['JKS', 'J', 'JO', 'JK', 'JKC', 'JKG', 'JKB', 'JKV', 'JKQ',
              'JX', 'JC', 'JKI', 'JKO', 'JKM', 'ETM']
  preprocessed_text = []
  for text in context:
      morphs = mecab.pos(text, join=False)

      # 존재하는지에 대한 여부
      nn_flag = False
      v_flag = False
      j_flag = False
      for morph in morphs:
        pos_tags = morph[1].split("+")   # ('신다', 'EP+EC') -> ['EP', 'EC']
        for pos_tag in pos_tags:
          if not nn_flag and pos_tag in NN_TAGS:  # True면 False
              nn_flag = True
          if not v_flag and pos_tag in V_TAGS:
              v_flag = True
          if not j_flag and pos_tag in J_TAGS:
              j_flag = True
        if nn_flag and v_flag and j_flag:   # 모두 True라면
          preprocessed_text.append(text)
          break
  return preprocessed_text


In [143]:
def min_max_filter(min_len, max_len, context):
      filtered_text = []

      for text in context:
        if min_len <= len(text) and len(text) < max_len:
          filtered_text.append(text)

      return filtered_text


In [144]:
for index, row in df.iterrows():
    title = row['뉴스제목']  # 엑셀 파일의 '제목' 컬럼
    content = row['본문']   # 엑셀 파일의 '본문' 컬럼

    # article 객체를 가정하고 title과 text에 할당
    article = type('', (), {})()  # 빈 객체 생성
    article.title = title
    article.text = content

    print('title:', article.title)
    print('content:', article.text)

    context = article.text.split('\n')

    context_with_index = [(i, text) for i, text in enumerate(context)]

    processed_context = delete_stopwords(context)

    processed_context_with_index = [(i, text) for i, text in enumerate(processed_context)]

    processed_context = delete_html_tag(processed_context)

    processed_context_with_index = [(i, text) for i, text in enumerate(processed_context)]

    splited_context = sentence_separator(processed_context)

    splited_context_with_index = [(i, text) for i, text in enumerate(splited_context)]

    processed_context = delete_email(splited_context)

    processed_context_with_index = [(i, text) for i, text in enumerate(processed_context)]

    processed_context = delete_hashtag(processed_context)

    processed_context_with_index = [(i, text) for i, text in enumerate(processed_context)]

    processed_context = delete_copyright(processed_context)

    processed_context_with_index = [(i, text) for i, text in enumerate(processed_context)]

    normalized_context = duplicated_sentence_normalizer(processed_context)

    normalized_context_with_index = [(i, text) for i, text in enumerate(normalized_context)]

    post_processed_context = morph_filter(normalized_context)

    post_processed_context = min_max_filter(20, 60, post_processed_context) # 20글자 이상 60글자 미만의 문장들만 반환

    # fit_on_texts: 토큰끼리 분리하여 문장을 처리하여 학습
    tokenizer.fit_on_texts(post_processed_context)

    # word_index: 토큰화된 토큰들이 인덱스화됨
    word2idx = tokenizer.word_index  # 토큰화된 토큰들이 인덱스화됨

    # 토큰 인코딩
    encoded = tokenizer.texts_to_sequences(post_processed_context)
    print(encoded)

    print()

title: "BTS 전역만 기다렸는데"…'설상가상' 하이브 개미들 '비명'..
content: '어도어 사태'·표절 의혹에 하이브 투자심리 위축
2분기 실적도 부진할 것으로 예상돼

"하반기 아티스트 활동 본격화하며 실적 개선될 것"
"내년 BTS 완전체 활동 기대"…내년 6월엔 모든 멤버 전역
배임 혐의로 경찰조사를 마친 민희진 어도어 대표가 지난 9일 오후 서울 용산경찰서에서 취재진의 질문에 답변하고 있다. 사진=뉴스1

하이브가 부진의 늪을 벗어나지 못하고 있다. 그룹 방탄소년단(BTS)의 멤버 진(본명 김석진)이 전역하며 활동을 시작했지만, '어도어 사태', '버블검 표절 논란'에 투자심리가 얼어붙었다. 게임 자회사 하이브IM도 아직 유의미한 성과를 내지 못하는 상태다. 증권가에선 BTS 완전체 활동이 가능한 내년을 주목해야 한다고 조언했다.

20일 한국거래소에 따르면 하이브는 지난 19일 장중 17만8700원까지 밀리며 52주 최저가를 새로 썼다. 작년 8월 8일 기록한 52주 최고가 29만500원에 비해 40%가량 낮은 수치다.
이날 주가는 전날보다 1.35% 하락한 18만2500원에 장을 마쳤다. 연초 10조원에 육박하던 시가총액도 7조6000억원대로 쪼그라들었다.

이달 들어 하이브는 2거래일만 상승세를 보이는 등 내리막길을 걷고 있다. '큰 손' 외국인과 기관 투자자가 하락세를 주도하는 모습이다. 이달 외국인은 하이브 주식 541억원어치를 순매도했다. 기관도 497억원어치를 팔아치웠다. 개인은 1038억원 순매수하며 물량을 받아내고 있다.
걸그룹 뉴진스(NewJeans)./사진=뉴스1

1~2월 약세를 보였던 하이브는 3월부터 실적 기대감 덕에 반등하기 시작했다. 하지만 '어도어 사태' 후 투자심리가 급격히 얼어붙었다. 4월 22일 하이브가 산하 레이블인 어도어 경영진에 대한 감사를 시작했다는 소식이 전해졌다. 당시 하이브는 하루 만에 7.81% 급락했고, 시총 7500억원이 증발했다.

어도어 사태는 현재진행형이다. 민 대표는 5월 말 어

In [145]:
for index, row in df.iterrows():
    title = row['뉴스제목']  # 엑셀 파일의 '제목' 컬럼
    content = row['본문']   # 엑셀 파일의 '본문' 컬럼

    # article 객체를 가정하고 title과 text에 할당
    article = type('', (), {})()  # 빈 객체 생성
    article.title = title
    article.text = content

    print('title:', article.title)
    print('content:', article.text)

    context = article.text.split('\n')

    context_with_index = [(i, text) for i, text in enumerate(context)]

    processed_context = delete_stopwords(context)

    processed_context_with_index = [(i, text) for i, text in enumerate(processed_context)]

    processed_context = delete_html_tag(processed_context)

    processed_context_with_index = [(i, text) for i, text in enumerate(processed_context)]

    splited_context = sentence_separator(processed_context)

    splited_context_with_index = [(i, text) for i, text in enumerate(splited_context)]

    processed_context = delete_email(splited_context)

    processed_context_with_index = [(i, text) for i, text in enumerate(processed_context)]

    processed_context = delete_hashtag(processed_context)

    processed_context_with_index = [(i, text) for i, text in enumerate(processed_context)]

    processed_context = delete_copyright(processed_context)
    # 여기부터추가부터
    normalized_context = duplicated_sentence_normalizer(processed_context)

    post_processed_context = morph_filter(normalized_context)

    post_processed_context = min_max_filter(20, 60, post_processed_context)
    # 추가 여기까지
    processed_context_with_index = [(i, text) for i, text in enumerate(processed_context)]

    normalized_context = duplicated_sentence_normalizer(processed_context)

    normalized_context_with_index = [(i, text) for i, text in enumerate(normalized_context)]

    post_processed_context = morph_filter(normalized_context)

    post_processed_context = min_max_filter(20, 60, post_processed_context) # 20글자 이상 60글자 미만의 문장들만 반환

    # 토큰화
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(post_processed_context)

    # word_index: 토큰화된 토큰들이 인덱스화됨
    word2idx = tokenizer.word_index

    # 토큰 인코딩
    encoded = tokenizer.texts_to_sequences(post_processed_context)
    print(encoded)

    # 임베딩
    vocab_size = len(word2idx) +1
    print(f'단어 사전의 크기는: {vocab_size}개')

title: "BTS 전역만 기다렸는데"…'설상가상' 하이브 개미들 '비명'..
content: '어도어 사태'·표절 의혹에 하이브 투자심리 위축
2분기 실적도 부진할 것으로 예상돼

"하반기 아티스트 활동 본격화하며 실적 개선될 것"
"내년 BTS 완전체 활동 기대"…내년 6월엔 모든 멤버 전역
배임 혐의로 경찰조사를 마친 민희진 어도어 대표가 지난 9일 오후 서울 용산경찰서에서 취재진의 질문에 답변하고 있다. 사진=뉴스1

하이브가 부진의 늪을 벗어나지 못하고 있다. 그룹 방탄소년단(BTS)의 멤버 진(본명 김석진)이 전역하며 활동을 시작했지만, '어도어 사태', '버블검 표절 논란'에 투자심리가 얼어붙었다. 게임 자회사 하이브IM도 아직 유의미한 성과를 내지 못하는 상태다. 증권가에선 BTS 완전체 활동이 가능한 내년을 주목해야 한다고 조언했다.

20일 한국거래소에 따르면 하이브는 지난 19일 장중 17만8700원까지 밀리며 52주 최저가를 새로 썼다. 작년 8월 8일 기록한 52주 최고가 29만500원에 비해 40%가량 낮은 수치다.
이날 주가는 전날보다 1.35% 하락한 18만2500원에 장을 마쳤다. 연초 10조원에 육박하던 시가총액도 7조6000억원대로 쪼그라들었다.

이달 들어 하이브는 2거래일만 상승세를 보이는 등 내리막길을 걷고 있다. '큰 손' 외국인과 기관 투자자가 하락세를 주도하는 모습이다. 이달 외국인은 하이브 주식 541억원어치를 순매도했다. 기관도 497억원어치를 팔아치웠다. 개인은 1038억원 순매수하며 물량을 받아내고 있다.
걸그룹 뉴진스(NewJeans)./사진=뉴스1

1~2월 약세를 보였던 하이브는 3월부터 실적 기대감 덕에 반등하기 시작했다. 하지만 '어도어 사태' 후 투자심리가 급격히 얼어붙었다. 4월 22일 하이브가 산하 레이블인 어도어 경영진에 대한 감사를 시작했다는 소식이 전해졌다. 당시 하이브는 하루 만에 7.81% 급락했고, 시총 7500억원이 증발했다.

어도어 사태는 현재진행형이다. 민 대표는 5월 말 어

In [146]:
# 임베딩
vocab_size = len(word2idx) +1
print(f'단어 사전의 크기는: {vocab_size}개')

단어 사전의 크기는: 70개
